In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

In [2]:
raw = pd.read_csv('reformatted_peg_data.csv',usecols=['Name','Date','East','North','Elev'])
raw = raw.rename(columns=str.lower)
raw['date'] = pd.to_datetime(raw['date'])

In [3]:
clean = pd.DataFrame()

groups = list(raw.groupby('name'))
for group, df in groups:
    df.sort_values('date',inplace=True,ignore_index=True)
    df = df[(df.east != 0) & (df.north != 0) & (df.elev != 0)].copy()
    
    df['e_disp'] = df['east'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
    df['n_disp'] = df['north'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
    df['v_disp'] = df['elev'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
    
    df['cum_e_disp'] = df['e_disp'].cumsum()
    df['cum_n_disp'] = df['n_disp'].cumsum()
    df['cum_v_disp'] = df['v_disp'].cumsum()
    
    df.iloc[0,5:] = 0
    
    clean = clean.append(df,ignore_index=True)
    
clean.sort_values(['name','date'],inplace=True,ignore_index=True)

In [4]:
gdf = gpd.GeoDataFrame(clean, geometry=gpd.points_from_xy(clean.east, clean.north),crs='EPSG:2193')

In [5]:
gdf = gdf.to_crs('EPSG:32760')

In [6]:
gdf['date'] = gdf.date.dt.strftime('%Y%m%d')

In [7]:
gdf['east'] = gdf.geometry.x
gdf['north'] = gdf.geometry.y

In [8]:
gdf.explore()

In [9]:
gdf.to_csv('pegs.csv',index=False)

In [10]:
gdf.to_file('pegs.shp')